In [1]:
#import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

### Working with images

In [2]:
###### read an image ####
def read(img_path):
   return cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

##### save an image ###
def save(img, img_path="output.jpeg"):
   cv2.imwrite(img_path, img)

In [3]:
### get a list of all the images in a folder ####

# returns (list of images, list of names)
def list_images(folder_path):
    result = []

    # get list of names of all the images in the folder
    file_names = os.listdir(folder_path)

    # read each image as numpy aaray
    for img_name in file_names:
        # full path of the img = folder_path + image name
        image_path = os.path.join(folder_path, img_name)

        # Read the image and add it to the list
        result.append(read(image_path))

    return (result, file_names)

In [4]:
#create a composite image from a list of images

# images = list of images
def get_composite(images):
    # Ensure all images have the same shape
    image_shape = images[0].shape
    for img in images:
        if img.shape != image_shape:
            raise ValueError("All images must have the same shape")

    # Initialize an array to store the composite image
    composite_image = np.zeros_like(images[0], dtype=np.float32)

    # Sum the pixel values from all images
    for img in images:
        composite_image += img.astype(np.float32)

    # Normalize the composite image to the range [0, 255]
    composite_image = (composite_image / len(images)).astype(np.uint8)

    return composite_image

## Denoising

##### Denoising (Adaptive non-local means)

In [5]:
###### adaptive NLM #####
def denoise(img, h=10, window=100, patch=7):
    result = cv2.fastNlMeansDenoising(img, None, h=h, templateWindowSize=patch, searchWindowSize=window)

    return result

##### Contrast enhancement (custom)

In [6]:
############# contrast enhancement ##################
def contrast_enhancement(img, threshold=10, box=(2,4)):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = img.copy()

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # denoise the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                #make the black pixels blacker
                patch[patch <= threshold] = patch[patch <= threshold]*(0.8)
                
                # make the non-black pixels black
                patch[patch > threshold] = avg

            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result

##### Final function

In [7]:
def denoise_image(img, box=(2,4)):
    d = denoise(img)
    d1 = contrast_enhancement(d, threshold=10, box=box)
    d2 = contrast_enhancement(d, threshold= 30, box=box)
    d3 = contrast_enhancement(d, threshold=45, box=box)
    #d4 = contrast_enhancement(d, threshold=45, box=box)
    
    images = [d1, d2, d3]

    result = get_composite(images)

    return result

## Layer segmentation

### Research paper-based methods

#### SIN filtering

In [71]:
def SIN(img):
    # normalise ?????
    img = np.float32(img) / 255.0

    #initialise result image
    result = np.zeros_like(img)

    # Apply SIN filter
    for i in range(img.shape[0]):
        result[i, :] = np.sin(img[i, :])

    # Convert image back to uint8
    result = np.uint8(result * 255.0)

    return result

#### Self fusion

In [73]:
import torch
import torch.nn as nn

In [75]:
class SelfFusionNet(nn.Module):
    def __init__(self):
        super(SelfFusionNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        out = torch.add(out, x)
        return out

def self_fusion_NN(img_path, model_path):
    # Load OCT B-scan image
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = np.float32(img) / 255.0

    # Load self-fusion neural network model
    model = torch.jit.load(model_path)

    # Convert image to tensor
    img_tensor = torch.from_numpy(img).unsqueeze(0).unsqueeze(0)

    # Apply self-fusion neural network
    with torch.no_grad():
        denoised_tensor = model(img_tensor)
    denoised_img = denoised_tensor.squeeze().numpy()

    # Convert image back to uint8
    denoised_img = np.uint8(denoised_img * 255.0)

    return denoised_img

### My subtraction method (experimenting)

##### Smoothening

For a patch of size (w,h), make all the pixels in patch equal to the patch average. Do this for both (w,h) and (h,w)

In [8]:
########### smoothening the layers #####

#smoothen uni directionally
def smoothen_uni(img, box=(1,7)):
    # shape of image , box
    rows, cols = img.shape #image
    box_width, box_height = box[1], box[0] #box

    #create the the result array  
    result = img.copy() #same shape as input

    #process the image patch by patch
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            #make all pixels equal to average
            patch[:, :] = avg
                
            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result


def smoothen(img, box=(2,5)):
    box2 = [box[1], box[0]]

    t1 = smoothen_uni(img, box)
    t2 = smoothen_uni(img, box2)

    result = get_composite([t1, t2])

    return result

##### Enhance the image 

In [9]:
######## enhance ###############
''' Make all black pixels 0, and all white pixels 255. Leave the grey pixels as it is.'''

def process(img):
    # shape of image , box
    rows, cols = img.shape #image

    #create the the result array  
    result = img.copy() #same shape as input

    #calulate the threshholds
    # black = avg pixel value for the entire image
    tb = np.mean(img)
    # grey threshhold = avg value of non-black pixels
    grey_pixels = img[img > tb] #arr of all non-blk pixels
    tg = np.mean(grey_pixels)
    
    # go through the img pixel by pixel
    for y in range(rows):
        for x in range(cols):
            pixel = img[y, x]

            # make black pixels 0
            if pixel <= tb:
                result[y, x] = 0
            #keep grey pixels as it is
            #make white pixels 255
            elif pixel > tg:
                result[y, x] = 255

    return result



##### Subtract

From each pixel, subtract the value of the pixel above it.

In [10]:
def subtract(img):
    # Create a copy of the input image to store the result
    result = img.copy()

    # Convert the image data to a 32-bit signed integer data type
    # to avoid overflow
    img = img.astype(np.int32)

    # Calculate the height and width of the image
    height, width = img.shape

    for y in range(1, height):
        for x in range(width):
            # Subtract the value of the pixel above from the current pixel
            diff = img[y, x] - img[y-1, x]
            # Apply ReLU activation
            result[y, x] = max(0, diff)

    return result

##### Brighten

Make the picture brighter by increasing the value of each pixel k times.

In [11]:
def brighten(img, k=3):
    # Create a copy of the input image to store the result
    result = img.copy()

    # Brighten the image by multiplying each pixel value by 3
    result = result * k

    # Clip values to ensure they are within the valid range [0, 255]
    result = np.clip(result, 0, 255)

    return result.astype(np.uint8)

##### Finally get the boundaries

In [12]:
def get_boundaries(img):
    temp = smoothen(img, box=(2,5))
    temp = subtract(temp)
    result = brighten(temp)

    return result


## Testing

#### Folders

In [13]:
images, names = list_images(".\\denoised")

count=0
for img in images:
    result = get_boundaries(img) #some function to process the image
    save(result, f".\\output\\{names[count]}.jpg")
    count +=1 


#### Individual images

In [126]:
# orig = read(".\\originals\\f-180.jpg")
# img = denoise_image(orig)

img = read(".\\denoised\\f-180.jpg.jpg") #denoised

result = get_boundaries(img)

save(result, "output.jpg")